# import notebook?

How to execute and import a jupyter notebook?



How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?

TODO: not allowed to import cells that are not fully validated and unit tested.


## importNotebook(search)


#### the code



In [1]:
// initialize
var path = require('path');
var fs = require('fs')

// TODO: combine with id2 from rpc and nicename from notebook export
// TODO: insert niceName and getExports here?
// must have a unique id for each unique cell so that
// individual cells can serve as modules as well as notebooks
// adding the cell id as a part of the filename
function getCellPath(cell) {
    var question = cell.questions && cell.questions[0]
        ? (' aka ' + cell.questions[0].substr(0, 50))
        : '';
    return path.join(path.dirname(path.resolve(cell.filename)), cell.id) + question
}

// How to test if a notebook has already been imported
function importNotebook(notebook, ctx = {}) {
    if (typeof notebook === 'undefined') {
        return Promise.resolve({});
    }
    // read notebook from files and make a module
    var cells;
    if(notebook[0] === '.'
       && fs.existsSync(path.resolve(__dirname, notebook))) {
        cells = getCells(path.resolve(__dirname, notebook))
    } else {
        cells = interpret(notebook)
    }

    if(typeof cells.code !== 'undefined') {
        
        return makeModule(cells.source.join(''),
                          getCellPath(cells),
                          ctx);
        
    }
    console.log(`importing ${notebook} - ${cells.length} cells`)
    return cells.reduce((obj, c, i) => {
        
        var result = makeModule(c.source.join(''),
                                getCellPath(c),
                                ctx);
        
        if(typeof result === 'object') {
            if(typeof result[Object.keys(result)[0]] === 'function') {
                const func = result[Object.keys(result)[0]];
                obj[cells[i].id] = func;
                obj[func.name] = obj[cells[i].id];
            }
            Object.assign(obj, result)
        }
        if (typeof result === 'function') {
            obj[cells[i].id] = result;
            obj[result.name] = obj[cells[i].id];
        }
        obj[i] = result;
        Object.assign(ctx, obj);
        return obj;
    }, {});
}

module.exports.import = importNotebook;

// $$.done(); for our dumb parser



## makeModule(code, path, context)

Imports arbitrary code as a Node module.


#### make a module from a string of code



Import arbitrary code in to the current context?

Import arbitrary code in to a new context?


## options

* useCache: true
* __filename: path or filename in errors
* 


TODO: webpack other languages and import

TODO: use zeromq to import any language using other kernels

TODO: use language server from VS Code


#### the code

how does node module require work?

How to turn jupyter notebooks in to modules?


In [ ]:
// initialize

var Module = require('module').Module
var path = require('path')

function getCached(filepath) {
    // must have a new name for every generation otherwise cache will be returned
    var mtime = fs.statSync(filepath
    // TODO: find a better way to serve notebook names
        .replace(/\.ipynb(\[[0-9]+\]).*$/ig, '.ipynb')).mtime.getTime();

    // TODO: don't use cache of parent modules that have includes that have changed
    var cachedModule = Module._cache[filepath];
    if (cachedModule && mtime < cachedModule.buildTime) {
        return cachedModule;
    }
}

function makeModule(code, pathToCode, ctxGlobal) {
    var filepath = path.resolve(process.cwd(), pathToCode);
    
    ctxGlobal = ctxGlobal || {}
    ctxGlobal.module = getCached(filepath)
    if(ctxGlobal.module) {
        return ctxGlobal.module.exports
    }
    Object.assign(ctxGlobal, {
        importer: require('../Core'),
        module: new Module(filepath, ctxGlobal.module),
        exports: {}
    })
    Object.assign(ctxGlobal.module, {
        module: ctxGlobal.module,
        exports: ctxGlobal.exports,
        parent: module,
        buildTime: (new Date()).getTime(),
        filename: pathToCode,
        paths: Module._nodeModulePaths(path.dirname(filepath))
    })
    Module._cache[filepath] = ctxGlobal.module;
    
    // this is basically the magic of the 'rewire' module, reuse this?
    const validVars = Object.keys(ctxGlobal)
        .filter(k => k.match(/^[a-z_][a-z0-9_]*$/i))
        .join(',');

    console.info('compiling ' + pathToCode);
    
    // TODO: use the transpiler or kernels here
    ctxGlobal.module._compile(`
module.exports = (({${validVars}}) => {
${code}
})`, filepath);
    
    // assign the actual imports internally by calling the func export
    var result = ctxGlobal.module.exports;
    ctxGlobal.exports = ctxGlobal.module.exports = {};
    result(ctxGlobal);
    ctxGlobal.module.loaded = true;
    ctxGlobal.module.exports = ctxGlobal.module.exports
        || ctxGlobal.exports || result;
    return ctxGlobal.module.exports;
}

module.exports.makeModule = makeModule;

// $$.done() for our dumb parser


## displayCell(id)


#### the code

How to display interpreted results in markdown?



In [ ]:
// initialize

function resultMarkdown(res) {
    return ('\n\n\n' + res.length + ' match'
        + (res.length !== 1 ? 'es' : '')
        + ' found: ' + res.join(' , ') + '\n\n\n'
        + (res.length > 0
            ? ('\n\n\n' + res.markdown.join('\n') + '\n\n\n'
                + '```\n\n\n' + res.code + '\n\n\n```\n\n\n')
            : ''));
};

function displayCell(results) {
    return (typeof results[0] !== 'undefined' && typeof results[0] !== 'string'
        ? results.reduce((str, res) => {
            str += resultMarkdown(res);
            return str;
        }, '')
        : resultMarkdown(results));
}

module.exports.displayCell = displayCell


// $$.done() for our dumb parser


## interpret(query)

How to answer questions using jupyter notebooks and simple search?

How to search jupter notebooks for questions and answers?


#### search notebooks using question marks as the index



This function has 4 modes:

```
search terms
search notebook.ipynb
search notebook.ipynb[0]
search notebook.ipynb[search terms]
```

Passing in a string returns the top match of a single cell, unless the string names a notebook in which case it returns and Array of all the cells in the notebook.
`[0]` returns the index of code cells, markdown cells are accumulated leading up to a code cell.
`[search terms]` searches the markdown within the specified notebook, it's garunteed to return a cell from the notebook or no match error.



#### the code

How to search notebook questions?


In [ ]:
// initialize

var path = require('path');
var fs = require('fs');
var Fuse = require('fuse.js');
var {
    cacheAll,
    cellCache,
    lookupCell
} = cache = importNotebook('./cache.ipynb')

var FUSE_CONFIG = {
    caseSensitive: false,
    findAllMatches: true,
    distance: 100,
    threshold: 0.8,
    tokenize: true,
    shouldSort: true,
    keys: ['2.questions'],
    id: '1'
}

var token = new Fuse(cellCache, FUSE_CONFIG);
var fuse = new Fuse(cellCache, Object.assign({}, FUSE_CONFIG, {
    tokenize: false
}));

var first = true;
function interpret(queries) {
    if(first) {
        first = false;
        cacheAll();
    }
    
    const fileMode = typeof queries === 'string' && queries.match(/\.ipynb$/ig);
    let searchResults = [].concat.apply([], (typeof queries === 'string'
        ? [queries]
        : queries).map(query => {
        query = path.basename(query).split(/[\[\]]/ig);
        var searchResults;
        if (query.length === 3) {
            searchResults = [searchQueryFiles(query)[0]]
        } else if (query[0].match(/\.ipynb$/ig)) {
            searchResults = searchFiles(query[0]);
        } else {
            searchResults = [queryDatabase(query[0])[0]];
        }
        if(searchResults.length === 0 || typeof searchResults[0] === 'undefined')
            throw new Error(`Nothing found for ${JSON.stringify(queries)
                            .substr(0, 200)}`);
        return searchResults
    }))

    return typeof queries === 'string' && !fileMode
        ? lookupCell(searchResults[0])
        : searchResults.map(r => lookupCell(r));
}

// filter query results by filename
function filterFilename(arr, fname) {
    return arr
        .filter(id => id.substr(0, fname.length + 1) === fname + '[');
}

function queryDatabase(search) {
    var tokenResults = token.search(search);
    var fuseResults = fuse.search(search);
    return fuseResults
        .filter(s => tokenResults.includes(s))
        .concat(tokenResults.filter(s => fuseResults.includes(s)))
}

function searchFiles(query) {
    return filterFilename(cellCache.map(c => c.id),
                          path.basename(query))
        .sort((a, b) => 
              parseInt(a.split(/[\[\]]/ig)[1])
              - parseInt(b.split(/[\[\]]/ig)[1]))
}

function searchQueryFiles(query) {
    var isNumeric = parseInt(query[1]) + '' === query[1];
    return isNumeric
        ? [cellCache.filter(cell => cell.id === `${query[0]}[${query[1]}]`)[0].id]
        : filterFilename(queryDatabase(query[1]), query[0])
}

module.exports.interpret = interpret;

// $$.done() for our dumb parser


## basic tools



### streamJson()

Because notebooks could store images and data sets. They might get fairly large, unfortunately. Stream the file so the big parts don't stick around in memory.

#### the code

How to stream json using a {match} function?

TODO: remove dependence on jsonpath, don't need matching, just the streaming part


In [ ]:
// initialize


// TODO: replace json path with this
function walkJson() {
    
}

function selectJson() {
    
}

var fs = require('fs');
var {JSONPath} = require('jsonpath-plus');
var fileCache = {};

function streamJson(file, path, match = ((c) => true)) {
    const json = JSON.parse(fs.readFileSync(file).toString());
    const nodes = JSONPath({json: Object.assign({}, json), path: '$..', resultType: 'all', callback: n => {
        if(typeof n.value !== 'object' || n.value === null) {
            return;
        }
        try {
            return match({
                path: n.path.split(/[\\'\]\[\$]+/igm).slice(1),
                value: n.value
            });
        } catch (e) {
            console.error(e);
            return;
        }
    }});
}

module.exports.streamJson = streamJson;

// $$.done() for our dumb parser


### regexToArray(exp, source[, index])

This should be built in to Javascript probably..


#### the code

How to convert regexp matches to an array?

In [ ]:
// initialize

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i])) ;
    return co;
}

module.exports.regexToArray = regexToArray;

// $$.done() for our dumb parser


### runAllPromises(promises[])

Sometimes you want to run promises in series instead of in parallel.


#### the code

How to run all promises sequentially?



In [ ]:
// initialize

function runAllPromises(promises) {
    return promises.reduce((promise, func) => {
        return promise.then(result => {
            return (typeof func == 'function'
                ? (new Promise(func)) : Promise.resolve(func))
                .then(Array.prototype.concat.bind(result));
        });
    }, Promise.resolve([]));
}

module.exports.runAllPromises = runAllPromises;

// $$.done() for our dumb parser



### getCells(notebook)



#### the code

How to parse cells from a notebook?



In [ ]:
// initialize

var path = require('path')

function getCells(notebook, types = ['*', 'code']) {
    notebook = path.resolve(notebook)
    //console.info('reading notebook ' + notebook);
    var json = JSON.parse(fs.readFileSync(notebook))
    var kernel = json.metadata.kernelspec;
    var cells = json.cells
        .filter(c => types.includes(c.cell_type))
        .map((c, i) => Object.assign(c, {
            language: kernel.language,
            filename: notebook,
            id: `${path.basename(notebook)}[${i}]`
        }));
    return cells
}

module.exports.getCells = getCells;


// $$.done() for our dumb parser
